In [17]:
import os
import shutil
import random
import xml.etree.ElementTree as ET
from PIL import Image 

# === 原始資料來源 ===
xml_dir = "/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/merged_annotation"
image_root = "/Users/chia-huitsao/Downloads/HE-SICH-CT-IDS/SubdatasetB_JPG/set"

# === 目標 YOLO 資料夾結構 ===
base_dir = "/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset"
for subset in ["train", "val"]:
    os.makedirs(f"{base_dir}/images/{subset}", exist_ok=True)
    os.makedirs(f"{base_dir}/labels/{subset}", exist_ok=True)

# === 抓取 XML 檔名（不含副檔名）===
xml_filenames = [f[:-4] for f in os.listdir(xml_dir) if f.endswith(".xml")]

# === 隨機切分 80% train / 20% val ===
random.seed(42)
random.shuffle(xml_filenames)
split_idx = int(0.8 * len(xml_filenames))
train_list = xml_filenames[:split_idx]
val_list = xml_filenames[split_idx:]

def convert_and_copy(filelist, subset):
    for name in filelist:
        xml_path = os.path.join(xml_dir, name + ".xml")
        try:
            tree = ET.parse(xml_path)
            root = tree.getroot()

            # 找對應圖片
            img_path = None
            for rootdir, _, files in os.walk(image_root):
                if name + ".jpg" in files:
                    img_path = os.path.join(rootdir, name + ".jpg")
                    break

            if img_path is None:
                print(f"[SKIP] 找不到圖片: {name}.jpg")
                continue

            # 取得圖片大小
            with Image.open(img_path) as im:
                width, height = im.size

            # 轉換成 YOLO 格式
            label_lines = []
            for bbox in root.findall(".//bndbox"):
                xmin = int(bbox.find("xmin").text)
                ymin = int(bbox.find("ymin").text)
                xmax = int(bbox.find("xmax").text)
                ymax = int(bbox.find("ymax").text)

                x_center = ((xmin + xmax) / 2) / width
                y_center = ((ymin + ymax) / 2) / height
                box_w = (xmax - xmin) / width
                box_h = (ymax - ymin) / height

                label_lines.append(f"0 {x_center:.6f} {y_center:.6f} {box_w:.6f} {box_h:.6f}")

            # 複製圖片與標註
            shutil.copy(img_path, f"{base_dir}/images/{subset}/{name}.jpg")
            with open(f"{base_dir}/labels/{subset}/{name}.txt", "w") as f:
                if label_lines:
                    f.write("\n".join(label_lines))
                # 沒 bbox 就寫空白檔

        except Exception as e:
            print(f"[ERROR] {name}: {e}")

# === 執行轉換與搬移 ===
convert_and_copy(train_list, "train")
convert_and_copy(val_list, "val")

print("✅ 完成！所有圖片與 YOLO 標註已建立（含空白 txt）")

[SKIP] 找不到圖片: 013_05.jpg
[SKIP] 找不到圖片: 013_06.jpg
[SKIP] 找不到圖片: 013_00.jpg
[SKIP] 找不到圖片: 013_23.jpg
[SKIP] 找不到圖片: 013_09.jpg
[SKIP] 找不到圖片: 013_02.jpg
[SKIP] 找不到圖片: 013_19.jpg
[SKIP] 找不到圖片: 013_18.jpg
[SKIP] 找不到圖片: 013_11.jpg
[SKIP] 找不到圖片: 013_12.jpg
[SKIP] 找不到圖片: 013_16.jpg
[SKIP] 找不到圖片: 013_10.jpg
[SKIP] 找不到圖片: 013_24.jpg
[SKIP] 找不到圖片: 013_27.jpg
[SKIP] 找不到圖片: 013_13.jpg
[SKIP] 找不到圖片: 013_26.jpg
[SKIP] 找不到圖片: 013_28.jpg
[SKIP] 找不到圖片: 013_29.jpg
[SKIP] 找不到圖片: 013_07.jpg
[SKIP] 找不到圖片: 013_08.jpg
[SKIP] 找不到圖片: 013_01.jpg
[SKIP] 找不到圖片: 013_14.jpg
[SKIP] 找不到圖片: 013_30.jpg
[SKIP] 找不到圖片: 013_20.jpg
[SKIP] 找不到圖片: 013_31.jpg
[SKIP] 找不到圖片: 013_17.jpg
[SKIP] 找不到圖片: 013_15.jpg
[SKIP] 找不到圖片: 013_25.jpg
[SKIP] 找不到圖片: 013_03.jpg
[SKIP] 找不到圖片: 013_21.jpg
[SKIP] 找不到圖片: 013_22.jpg
[SKIP] 找不到圖片: 013_04.jpg
✅ 完成！所有圖片與 YOLO 標註已建立（含空白 txt）


In [19]:
from pathlib import Path

# 修改成你的完整資料集根路徑
yolo_dataset_path = Path("/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset")

data_yaml = f"""
path: {yolo_dataset_path}
train: images/train
val: images/val
names:
  0: hemorrhage
"""

# 輸出到指定位置
output_path = yolo_dataset_path / "data.yaml"
with open(output_path, "w") as f:
    f.write(data_yaml.strip())

print(f"✅ data.yaml 已產生於：{output_path}")


✅ data.yaml 已產生於：/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/data.yaml


In [21]:
pip install tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 9.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 2.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 11.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from ultralytics import YOLO
import pandas as pd
import shutil
import os

# 載入 YOLOv8 的模型（可選：yolov8n.pt, yolov8s.pt, yolov8m.pt...）
model = YOLO("yolov8n.pt")  # n=Nano，最輕量，適合 CPU 訓練

# 訓練模型
results = model.train(
    data="/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/data.yaml",  # ← 請確認路徑正確
    epochs=100,
    patience=20,
    imgsz=512,
    batch=4,
    device="cpu",
    project="runs_tensorboard",
    name="task1_yolo",
    verbose=True
)

# === 將每個 epoch 的 metric 結果儲存為 CSV ===
#metrics = results.metrics  # list of dicts (每個 epoch 的訓練結果)
#df = pd.DataFrame(metrics)

# 儲存 CSV 路徑
#csv_path = os.path.join(results.save_dir, "/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/training_log.csv")
#df.to_csv(csv_path, index=False)

#print(f"📄 訓練歷史已儲存到：{csv_path}")




# 原始模型檔案路徑
#src = "runs/detect/train/weights/best.pt"

# 指定你要儲存的位置（請換成你自己的目標資料夾）
#dst = "/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolov8n_hemorrhage.pt"

# 複製模型
#shutil.copy(src, dst)

#print(f"✅ 模型已儲存到：{dst}")

New https://pypi.org/project/ultralytics/8.3.101 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.100 🚀 Python-3.12.4 torch-2.6.0 CPU (Apple M3)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/data.yaml, epochs=100, time=None, patience=20, batch=4, imgsz=512, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=runs_tensorboard, name=task1_yolo2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, re

train: Scanning /Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/labels/train.cache... 2784 images, 2205 backgrounds, 0 corrupt: 100%|██████████| 2784/2784 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/labels/val.cache... 695 images, 554 backgrounds, 0 corrupt: 100%|██████████| 695/695 [00:00<?, ?it/s]

Plotting labels to runs_tensorboard/task1_yolo2/labels.jpg... 


...

                   all        695        141      0.951      0.972      0.987      0.708

100 epochs completed in 9.056 hours.
Optimizer stripped from runs_tensorboard/task1_yolo2/weights/last.pt, 6.2MB
Optimizer stripped from runs_tensorboard/task1_yolo2/weights/best.pt, 6.2MB

Validating runs_tensorboard/task1_yolo2/weights/best.pt...
Ultralytics 8.3.100 🚀 Python-3.12.4 torch-2.6.0 CPU (Apple M3)


Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 87/87 [00:28<00:00,  3.09it/s]


                   all        695        141      0.952      0.977       0.99      0.715
Speed: 0.4ms preprocess, 37.3ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs_tensorboard/task1_yolo2


AttributeError: 'DetMetrics' object has no attribute 'metrics'. See valid attributes below.

    Utility class for computing detection metrics such as precision, recall, and mean average precision (mAP).

    Attributes:
        save_dir (Path): A path to the directory where the output plots will be saved.
        plot (bool): A flag that indicates whether to plot precision-recall curves for each class.
        names (dict): A dictionary of class names.
        box (Metric): An instance of the Metric class for storing detection results.
        speed (dict): A dictionary for storing execution times of different parts of the detection process.
        task (str): The task type, set to 'detect'.
    

In [32]:
import pandas as pd
import matplotlib.pyplot as plt

# 修改為你實際的訓練輸出資料夾
log_path = "/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/results.csv"

# 讀取訓練紀錄
df = pd.read_csv(log_path)

# 找最佳 mAP epoch
best_map_epoch = df["metrics/mAP50(B)"].idxmax()
best_map = df.loc[best_map_epoch, "metrics/mAP50(B)"]
early_stop_epoch = df["epoch"].max() + 1

# 畫圖
plt.figure(figsize=(12, 6))

# Loss 曲線
plt.subplot(1, 2, 1)
plt.plot(df["epoch"], df["train/box_loss"], label="Box Loss", marker="o")
plt.plot(df["epoch"], df["train/cls_loss"], label="Class Loss", marker="x")
plt.axvline(early_stop_epoch, color="red", linestyle="--", label="Early Stop")
plt.title("Training Loss Curve")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.legend()

# mAP 曲線
plt.subplot(1, 2, 2)
plt.plot(df["epoch"], df["metrics/mAP50(B)"], label="mAP@0.5", marker="o")
plt.plot(df["epoch"], df["metrics/mAP50-95(B)"], label="mAP@0.5:0.95", marker="x")
plt.axvline(best_map_epoch, color="green", linestyle="--", label=f"Best mAP Epoch: {best_map_epoch}")
plt.axvline(early_stop_epoch, color="red", linestyle="--", label="Early Stop")
plt.title("mAP Performance Curve")
plt.xlabel("Epoch")
plt.ylabel("mAP")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.savefig("/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/training_performance_summary.png")
plt.show()

print(f"✅ Best mAP@0.5: {best_map:.4f} at epoch {best_map_epoch}")
print(f"⏹️ Early Stopping at epoch {early_stop_epoch}")

<Figure size 1200x600 with 2 Axes>

✅ Best mAP@0.5: 0.9926 at epoch 69
⏹️ Early Stopping at epoch 101


In [60]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import cv2
import os
from pathlib import Path



model = YOLO("/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/best.pt")

val_images_dir = Path("/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/images/val")
label_dir = Path("/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/labels/val")
save_dir = "/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/task1_predictions"
os.makedirs(save_dir, exist_ok=True)

def visualize_one_image(hemorrhage=True):
    if hemorrhage:
        target_label = 1
        save_name = "task2_sample_hemorrhage.jpg"
    else:
        target_label = 0
        save_name = "task2_sample_non_hemorrhage.jpg"


    for txt_file in sorted(label_dir.glob("*.txt")):
        with open(txt_file) as f:
            lines = f.readlines()
        if hemorrhage and lines:
            # 有出血 → 至少有一行 bbox
            image_file = val_images_dir / (txt_file.stem + ".jpg")
            break
        elif not hemorrhage and not lines:
            # 無出血 → 空白 txt
            image_file = val_images_dir / (txt_file.stem + ".jpg")
            break
    else:
        print("找不到對應圖片")
        return


    # 預測
    pred_result = model(str(image_file))[0]
    pred_image = pred_result.plot()

    # 原圖
    original = cv2.imread(str(image_file))
    original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)

    # 顯示 + 儲存
    plt.rcParams['font.family'] = 'Heiti TC'
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(original)
    axes[0].set_title("原始圖片")
    axes[0].axis("off")

    axes[1].imshow(pred_image)
    axes[1].set_title("預測結果")
    axes[1].axis("off")

    plt.tight_layout()
    save_path = os.path.join(save_dir, save_name)
    plt.savefig(save_path)
    plt.show()
    print(f"✅ 圖片已儲存：{save_path}")


    # 執行兩張圖
print("\n顯示有出血的圖片預測：")
visualize_one_image(hemorrhage=True)

print("\n顯示無出血的圖片預測：")
visualize_one_image(hemorrhage=False)



顯示有出血的圖片預測：

image 1/1 /Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/images/val/0001_12.jpg: 512x512 (no detections), 53.9ms
Speed: 0.9ms preprocess, 53.9ms inference, 0.3ms postprocess per image at shape (1, 3, 512, 512)


<Figure size 1000x500 with 2 Axes>

✅ 圖片已儲存：/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/task1_predictions/task2_sample_hemorrhage.jpg

顯示無出血的圖片預測：

image 1/1 /Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/images/val/0001_07.jpg: 512x512 (no detections), 42.0ms
Speed: 0.5ms preprocess, 42.0ms inference, 0.3ms postprocess per image at shape (1, 3, 512, 512)


<Figure size 1000x500 with 2 Axes>

✅ 圖片已儲存：/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/task1_predictions/task2_sample_non_hemorrhage.jpg


In [ ]:

#####################################error  didn't fix################################################

from ultralytics import YOLO
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import cv2
import os
from pathlib import Path



model = YOLO("/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/best.pt")

val_images_dir = Path("/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/images/val")
label_dir = Path("/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/labels/val")
save_dir = "/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/task1_predictions"
os.makedirs(save_dir, exist_ok=True)

def visualize_one_image(hemorrhage=True):
    if hemorrhage:
        target_label = 1
        save_name = "task1_sample_hemorrhage.jpg"
    else:
        target_label = 0
        save_name = "task1_sample_non_hemorrhage.jpg"


    for txt_file in sorted(label_dir.glob("*.txt")):
        with open(txt_file) as f:
            lines = f.readlines()
        if hemorrhage and lines:
            # 有出血 → 至少有一行 bbox
            image_file = val_images_dir / (txt_file.stem + ".jpg")
            break
        elif not hemorrhage and not lines:
            # 無出血 → 空白 txt
            image_file = val_images_dir / (txt_file.stem + ".jpg")
            break
    else:
        print("找不到對應圖片")
        return
    
     # === 預測 ===
    pred_result = model(str(image_file), conf=0.001)[0]  # 👈 設定低信心閾值
    boxes = pred_result.boxes


    # 原圖 + 複製畫框用
    original = cv2.imread(str(image_file))
    original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    boxed = original.copy()

    if boxes is not None and len(boxes) > 0:
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # 邊界座標
            conf = float(box.conf[0])
            label_text = f"{conf:.2f}"
            # 畫框 + 信心值
            cv2.rectangle(boxed, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(boxed, label_text, (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
    else:
        print("⚠️ 沒有任何預測框")

    # 顯示與儲存
    plt.rcParams['font.family'] = 'Heiti TC'
    
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(original)
    axes[0].set_title("原始圖片")
    axes[0].axis("off")

    axes[1].imshow(boxed)
    axes[1].set_title("預測（全部框）")
    axes[1].axis("off")

    plt.tight_layout()
    save_path = os.path.join(save_dir, save_name)
    plt.savefig(save_path)
    plt.show()
    print(f"✅ 圖片已儲存：{save_path}")

    print(f"🔎 正在搜尋 {'有出血' if hemorrhage else '無出血'} 圖片...")
print("\n顯示有出血的圖片預測：")
visualize_one_image(hemorrhage=True)

print("\n顯示無出血的圖片預測：")
visualize_one_image(hemorrhage=False)


#####################################error  didn't fix################################################


顯示有出血的圖片預測：

image 1/1 /Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/images/val/0001_12.jpg: 512x512 (no detections), 62.7ms
Speed: 5.5ms preprocess, 62.7ms inference, 5.7ms postprocess per image at shape (1, 3, 512, 512)
⚠️ 沒有任何預測框


<Figure size 1000x500 with 2 Axes>

✅ 圖片已儲存：/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/task1_predictions/task1_sample_hemorrhage.jpg
🔎 正在搜尋 有出血 圖片...

顯示無出血的圖片預測：

image 1/1 /Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/images/val/0001_07.jpg: 512x512 (no detections), 49.0ms
Speed: 0.5ms preprocess, 49.0ms inference, 0.2ms postprocess per image at shape (1, 3, 512, 512)
⚠️ 沒有任何預測框


<Figure size 1000x500 with 2 Axes>

✅ 圖片已儲存：/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/task1_predictions/task1_sample_non_hemorrhage.jpg
🔎 正在搜尋 無出血 圖片...


In [59]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# 載入訓練好的模型（記得改成你自己訓練出的 best.pt 路徑）
model = YOLO("/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/best.pt")

# 載入你要推論的圖片
img_path = "/Users/chia-huitsao/Downloads/HE-SICH-CT-IDS/yolo_dataset/images/val/0004_18.jpg"  # <- 改成你的圖片路徑
results = model(img_path)

# 顯示 bounding box 結果
res_plotted = results[0].plot()  # 這會畫上 bounding boxes
plt.figure(figsize=(10, 10))
plt.imshow(res_plotted)
plt.axis("off")
plt.title("推論結果 - YOLOv8 出血區域") 
plt.show()
cv2.imwrite("/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_result_0004_18.jpg", res_plotted)
print("圖片已儲存為 yolo_result_0004_18.jpg")


image 1/1 /Users/chia-huitsao/Downloads/HE-SICH-CT-IDS/yolo_dataset/images/val/0004_18.jpg: 512x512 1 hemorrhage, 50.8ms
Speed: 1.3ms preprocess, 50.8ms inference, 6.1ms postprocess per image at shape (1, 3, 512, 512)


<Figure size 1000x1000 with 1 Axes>

圖片已儲存為 yolo_result_0004_18.jpg


In [ ]:
import pandas as pd
import os
from pathlib import Path

predictions = []

# 用你的 best.pt 模型
model = YOLO("/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/best.pt")

# 圖片資料夾（驗證集）
val_images_dir = Path("/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/images/val")

# 預測所有 val 圖片
for image_file in sorted(val_images_dir.glob("*.jpg")):
    results = model(str(image_file), conf=0.001)[0]  # 信心低也保留
    for box in results.boxes:
        x1, y1, x2, y2 = map(float, box.xyxy[0])
        conf = float(box.conf[0])
        cls_id = int(box.cls[0])
        predictions.append({
            "image": image_file.name,
            "class_id": cls_id,
            "confidence": round(conf, 4),
            "x1": round(x1, 1),
            "y1": round(y1, 1),
            "x2": round(x2, 1),
            "y2": round(y2, 1),
        })

# 輸出 CSV
df = pd.DataFrame(predictions)
output_path = "/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/task1_val_predictions.csv"
df.to_csv(output_path, index=False)
print(f"✅ 預測結果已儲存：{output_path}")

In [ ]:
import pandas as pd
import os
from pathlib import Path
from ultralytics import YOLO

predictions = []

model = YOLO("/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/best.pt")
val_images_dir = Path("/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/images/val")

for image_file in sorted(val_images_dir.glob("*.jpg")):
    results = model(str(image_file), conf=0.001)[0]
    boxes = results.boxes

    if boxes is not None and len(boxes) > 0:
        for box in boxes:
            x1, y1, x2, y2 = map(float, box.xyxy[0])
            conf = float(box.conf[0])
            cls_id = int(box.cls[0])
            predictions.append({
                "image": image_file.name,
                "class_id": cls_id,
                "confidence": round(conf, 4),
                "x1": round(x1, 1),
                "y1": round(y1, 1),
                "x2": round(x2, 1),
                "y2": round(y2, 1),
            })
    else:
        # 沒預測框 → 預測為「non-Hemorrhage」
        predictions.append({
            "image": image_file.name,
            "class_id": "none",        # ← 表示無出血
            "confidence": 0.0,
            "x1": None,
            "y1": None,
            "x2": None,
            "y2": None,
        })

# 輸出 CSV
df = pd.DataFrame(predictions)
output_path = "/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/tas2_val_predictions_with_none.csv"
df.to_csv(output_path, index=False)
print(f"✅ 預測結果（含 non-Hemorrhage）已儲存：{output_path}")



image 1/1 /Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/images/val/0001_07.jpg: 512x512 (no detections), 66.1ms
Speed: 5.1ms preprocess, 66.1ms inference, 5.7ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/images/val/0001_09.jpg: 512x512 (no detections), 39.6ms
Speed: 0.5ms preprocess, 39.6ms inference, 0.2ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/images/val/0001_11.jpg: 512x512 (no detections), 44.9ms
Speed: 0.5ms preprocess, 44.9ms inference, 0.2ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/images/val/0001_12.jpg: 512x512 (no detections), 35.4ms
Speed: 0.4ms preprocess, 35.4ms inference, 0.2ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /Users/chia-huitsao/Documents/PHE-SICH-CT

In [64]:
from ultralytics import YOLO
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# === Step 1: 預測驗證圖片 ===
model = YOLO("/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/best.pt")  # ← 換成你的模型路徑
val_images_dir = Path("/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/images/val")
label_dir = Path("/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/labels/val")

predictions = []
for image_file in sorted(val_images_dir.glob("*.jpg")):
    results = model(str(image_file), conf=0.001)[0]
    boxes = results.boxes

    if boxes is not None and len(boxes) > 0:
        for box in boxes:
            x1, y1, x2, y2 = map(float, box.xyxy[0])
            conf = float(box.conf[0])
            cls_id = int(box.cls[0])
            predictions.append({
                "image": image_file.name,
                "class_id": cls_id,
                "confidence": round(conf, 4),
                "x1": round(x1, 1),
                "y1": round(y1, 1),
                "x2": round(x2, 1),
                "y2": round(y2, 1),
            })
    else:
        predictions.append({
            "image": image_file.name,
            "class_id": "none",
            "confidence": 0.0,
            "x1": None,
            "y1": None,
            "x2": None,
            "y2": None,
        })

pred_df = pd.DataFrame(predictions)
pred_df["class_name"] = pred_df["class_id"].apply(lambda x: "Hemorrhage" if str(x) != "none" else "Non-Hemorrhage")
pred_df.to_csv("/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/task2_val_predictions_with_none.csv", index=False)
print("✅ 已儲存預測結果：task2_val_predictions_with_none.csv")

# === Step 2: 比對 Ground Truth 並分類 TP / FP / FN / TN ===
gt_dict = {}
for txt_file in sorted(label_dir.glob("*.txt")):
    with open(txt_file) as f:
        lines = f.readlines()
    gt_dict[txt_file.stem + ".jpg"] = 1 if lines else 0

records = []
pred_grouped = pred_df.groupby("image")

for image_name, group in pred_grouped:
    has_gt = gt_dict.get(image_name, 0)
    has_pred = (group["class_id"] != "none").any()

    if has_pred and has_gt:
        result = "TP"
    elif has_pred and not has_gt:
        result = "FP"
    elif not has_pred and has_gt:
        result = "FN"
    else:
        result = "TN"

    records.append({
        "image": image_name,
        "ground_truth": "Hemorrhage" if has_gt else "Non-Hemorrhage",
        "predicted": "Hemorrhage" if has_pred else "Non-Hemorrhage",
        "classification": result
    })

summary_df = pd.DataFrame(records)
summary_df.to_csv("/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/task2_val_classification_summary.csv", index=False)
print("✅ 已儲存分類對照表：task2_val_classification_summary.csv")

# === Step 3: 混淆矩陣 + 分類報告 ===
y_true = summary_df["ground_truth"]
y_pred = summary_df["predicted"]

# 混淆矩陣圖
cm = confusion_matrix(y_true, y_pred, labels=["Hemorrhage", "Non-Hemorrhage"])
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Hemorrhage", "Non-Hemorrhage"],
            yticklabels=["Hemorrhage", "Non-Hemorrhage"])
plt.xlabel("Predicted")
plt.ylabel("Ground Truth")
plt.title("任務一 - 混淆矩陣")
plt.tight_layout()
plt.savefig("/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/task2_confusion_matrix_summary.png")
plt.show()

# 分類報告（precision, recall, f1-score, accuracy）
print("\n📊 分類報告（Classification Report）：")
report = classification_report(y_true, y_pred, digits=4, output_dict=False)
print(report)

# 若你也想儲存成 txt：
with open("/Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/task2_classification_report.txt", "w") as f:
    f.write(classification_report(y_true, y_pred, digits=4))


image 1/1 /Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/images/val/0001_07.jpg: 512x512 (no detections), 48.3ms
Speed: 1.1ms preprocess, 48.3ms inference, 0.3ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/images/val/0001_09.jpg: 512x512 (no detections), 42.2ms
Speed: 0.5ms preprocess, 42.2ms inference, 0.2ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/images/val/0001_11.jpg: 512x512 (no detections), 33.7ms
Speed: 0.5ms preprocess, 33.7ms inference, 0.1ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /Users/chia-huitsao/Documents/PHE-SICH-CT-IDS/Hemorrhage_CT/yolo_dataset/images/val/0001_12.jpg: 512x512 (no detections), 37.7ms
Speed: 0.4ms preprocess, 37.7ms inference, 0.2ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /Users/chia-huitsao/Documents/PHE-SICH-CT

<Figure size 600x500 with 2 Axes>


📊 分類報告（Classification Report）：
                precision    recall  f1-score   support

    Hemorrhage     0.8917    0.9929    0.9396       141
Non-Hemorrhage     0.9981    0.9693    0.9835       554

      accuracy                         0.9741       695
     macro avg     0.9449    0.9811    0.9616       695
  weighted avg     0.9766    0.9741    0.9746       695

